In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from sklearn.metrics import classification_report

In [5]:
lob_data= pd.read_csv("C:/Users/keert/OneDrive/Desktop/Mini Project/resampled_lob_secs2.csv")
lob_data['Datetime'] = pd.to_datetime(lob_data['Datetime'])

# lob_data['price']=(lob_data['Max Bid Price']*lob_data['Max Bid Quantity']+lob_data['Min Ask Price']*lob_data['Min Ask Quantity'])/(lob_data['Max Bid Quantity']+lob_data['Min Ask Quantity'])
# lob_data
lob_data

,Datetime,Min Ask Price,Max Bid Price,price,Total Bid Quantity,Total Ask Quantity,Min Ask Quantity,Max Bid Quantity,ofi,RSI,MA,Momentum
0,2025-01-02 00:00:02,310,261,293.666667,7,7,4,2,6,NaN,NaN,NaN
1,2025-01-02 00:00:03,293,262,269.750000,7,13,2,6,8,NaN,NaN,NaN
2,2025-01-02 00:00:04,287,264,271.666667,7,14,3,6,9,NaN,NaN,NaN
3,2025-01-02 00:00:05,283,264,272.636364,9,14,5,6,5,NaN,NaN,NaN
4,2025-01-02 00:00:06,281,265,267.285714,18,16,1,6,7,8.976775,275.001082,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3767279,2025-07-01 08:17:55,183,109,133.666667,21,18,1,2,-1,79.558011,124.390476,35.666667
3767280,2025-07-01 08:17:56,183,110,128.250000,24,18,1,3,3,81.702128,131.840476,37.250000
3767281,2025-07-01 08:17:57,123,110,114.333333,26,18,1,2,0,34.012739,129.450000,-11.952381
3767282,2025-07-01 08:17:58,114,109,111.500000,25,19,1,1,0,20.289855,125.350000,-20.500000


In [6]:
# label:when the price increase 1% or decrease 1%, label=1,2, otherwise label=0
lob_data['label'] = 0
lob_data['label'][lob_data['price'].shift(-1) > lob_data['price']*1.01] = 1
lob_data['label'][lob_data['price'].shift(-1) < lob_data['price']*0.99] = 2

lob_data['label'].value_counts()

C:\Users\keert\AppData\Local\Temp\ipykernel_30548\3475079993.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lob_data['label'][lob_data['price'].shift(-1) > lob_data['price']*1.01] = 1
C:\Users\keert\AppData\Local\Temp\ipykernel_30548\3475079993.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lob_data['label'][lob_data['price'].shift(-1) < lob_data['price']*0.99] = 2


0    1677076
2    1187468
1     902740
Name: label, dtype: int64

In [7]:
lob_data['Datetime']=pd.to_datetime(lob_data['Datetime'])
lob_data['time_seconds']=lob_data['Datetime'].dt.hour*3600+lob_data['Datetime'].dt.minute*60+lob_data['Datetime'].dt.second

In [8]:
features = ['Max Bid Price', 'Max Bid Quantity', 'Min Ask Price', 'Min Ask Quantity', 'price', 'time_seconds','MA','Momentum','RSI']
lob_data.dropna(inplace=True)

split_index = int(0.9 * len(lob_data))
train_data = lob_data[:split_index]
test_data = lob_data[split_index:]


X_train = train_data[features]
y_train = train_data['label']

X_test = test_data[features]
y_test = test_data['label']

In [9]:
# scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)   

In [10]:
# 定义模型
model = xgb.XGBClassifier(
    objective='multi:softmax',  # 或 'multi:softprob' 如果你需要概率输出
    num_class=3,                # 设置类别总数
    # max_depth=20,                # 最大深度
    n_estimators=200  ,          # 树的个数
    n_jobs=-1
)
# 训练模型
model.fit(X_train_scaled, y_train)

# 预测
y_test_pred = model.predict(X_test_scaled)
y_train_pred = model.predict(X_train_scaled)

# 评估
print(classification_report(y_train, y_train_pred))
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.72      0.83      0.77   1583918
           1       0.55      0.31      0.40    786865
           2       0.60      0.65      0.62   1019768

    accuracy                           0.65   3390551
   macro avg       0.62      0.60      0.59   3390551
weighted avg       0.64      0.65      0.64   3390551

              precision    recall  f1-score   support

           0       0.56      0.34      0.42     93155
           1       0.46      0.60      0.52    115875
           2       0.64      0.65      0.65    167698

    accuracy                           0.56    376728
   macro avg       0.55      0.53      0.53    376728
weighted avg       0.57      0.56      0.55    376728



In [11]:
len(y_test_pred)

376728

In [37]:
def trading_simulator(prices, signals, initial_capital=10000, transaction_cost_rate=0.001, max_capital_per_trade_ratio=0.1):
    capital = initial_capital
    position = 0
    capital_history = []
    transaction_count = 0

    for price, signal in zip(prices, signals):
        if price <= 0:  # Prevent division by zero or negative prices
            continue

        transaction_cost = price * transaction_cost_rate
        max_capital_per_trade = capital * max_capital_per_trade_ratio

        if signal == 1 and capital >= (price + transaction_cost):  # Buy
            max_shares = int((max_capital_per_trade - transaction_cost) / price)
            shares = min(max_shares, max_shares)
            if shares > 0:
                capital -= (shares * price + transaction_cost)
                position += shares
                transaction_count += 1
                print(f"Bought {shares} shares at {price}, remaining capital: {capital}")

        elif signal == 2 and position > 0:  # Sell
            shares_to_sell = min(position, max_shares)
            revenue = shares_to_sell * price
            capital += revenue - (shares_to_sell * price * transaction_cost_rate)
            position -= shares_to_sell
            transaction_count += 1
            print(f"Sold {shares_to_sell} shares at {price}, new capital: {capital}")

        capital_history.append(capital + position * price if position > 0 else capital)

    final_portfolio_value = capital + position * prices[-1] if position > 0 else capital
    normalized_profit = (final_portfolio_value - initial_capital) / transaction_count if transaction_count > 0 else 0

    return final_portfolio_value, normalized_profit, capital_history

# Assume prices and signals are numpy arrays from your dataset
final_capital, normalized_profit, capital_history = trading_simulator(
    test_data['price'].values,
    y_test_pred,
    initial_capital=10000,
    transaction_cost_rate=0.001,
    max_capital_per_trade_ratio=0.1  # 10% of capital per trade
)

print(f"Final capital after trading: ${final_capital:.2f}")
print(f"Normalized profit per trade: ${normalized_profit:.2f}")


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Sold 0 shares at 160.5, new capital: 145.2926698372226
Sold 0 shares at 157.5, new capital: 145.2926698372226
Sold 0 shares at 145.8, new capital: 145.2926698372226
Sold 0 shares at 158.85714285714286, new capital: 145.2926698372226
Sold 0 shares at 108.2, new capital: 145.2926698372226
Sold 0 shares at 106.8, new capital: 145.2926698372226
Sold 0 shares at 136.0, new capital: 145.2926698372226
Sold 0 shares at 118.4, new capital: 145.2926698372226
Sold 0 shares at 114.0, new capital: 145.2926698372226
Sold 0 shares at 113.0, new capital: 145.2926698372226
Sold 0 shares at 112.0, new capital: 145.2926698372226
Sold 0 shares at 106.0, new capital: 145.2926698372226
Sold 0 shares at 105.66666666666669, new capital: 145.2926698372226
Sold 0 shares at 208.75, new capital: 145.2926698372226
Sold 0 shares at 184.0, new capital: 145.2926698372226
Sold 0 shares at 178.0, new capital: 145.2926698372226
Sold 0 shares at 160.75, new capital: 145.2926698372226
Sold 0 shares at 138.5, new capital: 